
### Final Project: Phase 2
Algorithms For Bioinformatics



**please change directory to folder that contain this code file and mitocondriy dataset folder.**




i code in colab. so i mount my drive. 

In [1]:
%cd "/content/drive/My Drive/phase2Algo"

/content/drive/My Drive/phase2Algo


In [2]:
mkdir 'blocks' #this folder create in the same path and contain blocks files

In [3]:
pip install biopython # for read fasta files

     |████████████████████████████████| 2.3MB 4.8MB/s 


In [4]:
pip install bitarray # for creating presence/absence table

     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for bitarray: filename=bitarray-1.4.2-cp36-cp36m-linux_x86_64.whl size=108510 sha256=180859b6f91f2d7ee2281580c88f06235a4656ffeb415a13031a9366776ec8e8
  Stored in directory: /root/.cache/pip/wheels/21/11/f3/7c9571443475b1a364ae541cf3cc80f9ceb4f6d6dc5ba7db44
Successfully built bitarray


In [5]:
from Bio import SeqIO
import os, glob
import itertools
import bitarray

In [6]:
def read_files(folder_path): 
    ''' reading files in given path

    Parameters
    --------
     - folder_path: path of files (type: str)
     
    Return
    --------
     - output: a dictionary that its keys are records id(genome name) and
               values are a string(genome or block) (type:dict)
    '''
    
    output = {}
    fasta_files = glob.glob(os.path.join(folder_path, '*.fasta')) 
    for fasta_file in fasta_files:
        for seq_record in SeqIO.parse(fasta_file, "fasta"):
            output.setdefault(seq_record.id,[]).append(str(seq_record.seq))
                 
    return output           

In [7]:
def split_genome(genome, nt_num, overlap = 0):
    ''' Split genome to blocks (phase 2.1)

    Parameters
    --------
     - genome: a genome (type: str)
     - nt_num: number of nucleotide in a block (type: int)
     - overlap: blocks overlap amount (type: int)

    Return
    --------
     - blocks (type: list)
    '''

    blocks = []
    start_index = 0
    while start_index < len(genome):
        block = genome[start_index : start_index + nt_num]
        start_index = start_index + nt_num - overlap  
        blocks.append(block)
    return blocks

In [8]:
def create_blocks(genomes):
    ''' create all blocks of all given genomes
    Parameters
    --------
     - genomes: all genomes (type: dict)
     
    Return
    --------
    write created blocks with specifict overlap(k) in "blocks" folder
    file name format: k + number of overlap + _ + genome name + .fasta (example: k0_AF010406.1.fasta)
    '''
    for genome in genomes:  
        for k in [0,3,5,7,9,11,24,32]:
              f_out = open('blocks/k'+str(k)+"_"+str(str(genome)+".fasta"), 'w')
              
              blocks = split_genome(genomes[genome][0] , nt_num = 1000 , overlap = k)

              f_out.write("; Blocks Count:" + str(len(blocks)) +"\n\n")

              block_num = 1
              for block in blocks:
                  f_out.write(">k" + str(k)+ "_" + str(genome)+ "\n" + block + "\n")
                  block_num += 1    
              f_out.close()

In [9]:
def initialize_bitarray(k):
    bit_array = bitarray.bitarray(4**k)
    bit_array.setall(False)
    return bit_array

In [10]:
def Count_Kmers(blocks , k):
    ''' Count kmers of blocks of given genome (phase 2.2)
    Parameters
    --------
     - blocks: all blocks given genome (type: list)
     - k: k(type: int)
    Return
    --------
     - Kmer_Count: counts of kmers in each block seprately (type: list of dics)
     - All_kmers: all kmers in each block (type: list)
    '''
    Kmer_Count = []
    All_kmers = []
    for block in blocks:
        kmers = []
        for i in range(len(block) - k + 1):
            kmer = block[i:i+k]
            kmers.append(kmer) 
        All_kmers.append(set(kmers))
        
        kmer_count = {}
        for kmer in kmers:
            kmer_count[kmer] = kmers.count(kmer)
        Kmer_Count.append(kmer_count)
    return All_kmers , Kmer_Count

In [11]:
def Filter(k, All_kmers):
    ''' Filter kmers by turn the index of presence kmers in related bit-array to 1 (phase 2.3)
    Parameters
    --------
     - k: kmers length (type: int)
     - All_kmers: kmers of blocks (type: list)
    Return
    --------
     - Presence_Absence: turn the index of present kmers in related bit-array (type: list of arrays)
    '''
    bm = { 'A' : 0, 'C' : 1, 'T' : 2, 'G' : 3 }
    Presence_Absence = []
  
    for block_kmers in All_kmers:
        ba = initialize_bitarray(k)
        for kmer in block_kmers:
            idx = 0
            for j in range(k-1, -1, -1):
                if kmer[j] not in ['A' , 'C', 'T', 'G']:
                    kmer = kmer.replace(kmer[j],'A')
                idx += 4**(k-j-1) * bm[kmer[j]]
            ba[idx] = True
        Presence_Absence.append(ba)
    return Presence_Absence

In [12]:
def Kmer_Count_and_Filter(All_blocks):
    ''' count kmers of each block of each genome and also filter them
    Parameters
    --------
     - All_blocks: all blocks of genome (type: list)
    Return
    --------
     - All_Counts: counts of kmers of all blocks(type: dict)
     - All_Presence_Absence: all presence/absence arrays (type: dict of lists)
    '''
    All_Counts = {}
    All_Presence_Absence = {}

    for genome in All_blocks:
          k = int(genome[genome.index('k')+1:genome.index('_')]) # be dast avardane k az file name
          if k != 0:
              all_kmers , kmer_counts = Count_Kmers(All_blocks[genome] , k)
              if k!= 24 and k!= 32: 
                 presence_absence_this_genome = Filter(k , all_kmers)
                 All_Presence_Absence[genome] = presence_absence_this_genome
              All_Counts[genome] = kmer_counts 
    return All_Counts ,All_Presence_Absence

In [13]:
def phase2_main():
    print("-> Read Genomes ...")
    genomes = read_files(folder_path = 'data mitochondrial genome')
    print("Genomes Readed!")
    print("-> Split Genomes to Blocks ...")
    create_blocks(genomes)
    print("Blocks Created!")
    All_blocks = read_files(folder_path = 'blocks')
    print("-> Start Count and Filter kmers ...")
    All_Counts ,All_Presence_Absence = Kmer_Count_and_Filter(All_blocks)
    print("Kmer Count and Filtering: Done!")
    return All_Counts ,All_Presence_Absence

In [14]:
All_Counts ,All_Presence_Absence = phase2_main()

-> Read Genomes ...
Genomes Readed!
-> Split Genomes to Blocks ...
Blocks Created!
-> Start Count and Filter kmers ...
Kmer Count and Filtering: Done!
